<a href="https://colab.research.google.com/github/Hebabeg/mlprojects/blob/main/Book_Recommendation_System_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

def generate_book_visualizations(books):
    """
    Generate comprehensive visualizations for book dataset

    Args:
        books (pd.DataFrame): Processed book dataset
    """

    # Convert 'Num_Ratings' to numeric, handling commas
    books['Num_Ratings'] = books['Num_Ratings'].str.replace(',', '', regex=True).astype(float)

    plt.figure(figsize=(20, 15))

    # 1. Correlation Matrix
    plt.subplot(2, 2, 1)
    numeric_columns = ['Avg_Rating', 'Num_Ratings']
    correlation_matrix = books[numeric_columns].corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5)
    plt.title('Rating and Number of Ratings Correlation')


    # 2. Genre Distribution
    plt.subplot(2, 2, 2)
    genre_counts = books['Genres'].str.split(',', expand=True).stack().value_counts().head(10)
    genre_counts.plot(kind='bar')
    plt.title('Top 10 Book Genres')
    plt.xticks(rotation=45, ha='right')

    # 3. Rating Distribution
    plt.subplot(2, 2, 3)
    books['Avg_Rating'].hist(bins=20)
    plt.title('Book Rating Distribution')
    plt.xlabel('Average Rating')
    plt.ylabel('Frequency')

    # 4. Author Frequency
    plt.subplot(2, 2, 4)
    author_counts = books['Author'].value_counts().head(10)
    author_counts.plot(kind='bar')
    plt.title('Top 10 Authors by Book Count')
    plt.xticks(rotation=45, ha='right')

    plt.tight_layout()
    plt.savefig('book_recommendation_visualizations.png')
    plt.close()

def analyze_book_data(books):
    """
    Provide detailed analysis of book dataset

    Args:
        books (pd.DataFrame): Processed book dataset

    Returns:
        dict: Dataset insights
    """
    insights = {
        'total_books': len(books),
        'unique_authors': books['Author'].nunique(),
        'unique_genres': len(set(genre for genres in books['Genres'].str.split(',') for genre in genres)),
        'avg_rating_stats': {
            'mean': books['Avg_Rating'].mean(),
            'median': books['Avg_Rating'].median(),
            'std': books['Avg_Rating'].std()
        },
        'rating_distribution': books['Avg_Rating'].value_counts(bins=5).to_dict()
    }

    return insights


generate_book_visualizations(books)
dataset_insights = analyze_book_data(books)
print(dataset_insights)


{'total_books': 10000, 'unique_authors': 6064, 'unique_genres': 1315, 'avg_rating_stats': {'mean': 4.068576999999999, 'median': 4.08, 'std': 0.3353586015150974}, 'rating_distribution': {Interval(4.0, 5.0, closed='right'): 5910, Interval(3.0, 4.0, closed='right'): 4057, Interval(2.0, 3.0, closed='right'): 22, Interval(-0.006, 1.0, closed='right'): 9, Interval(1.0, 2.0, closed='right'): 2}}


In [30]:
import gradio as gr
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import difflib

class BookRecommendationSystem:
    def __init__(self, books_df):
        self.books = books_df
        self.prepare_recommendation_system()

    def prepare_recommendation_system(self):

        self.books['Description'] = self.books['Description'].fillna('')
        self.books['Genres'] = self.books['Genres'].fillna('')
        self.books['Author'] = self.books['Author'].fillna('')

        books_data_features = (
            self.books['Description'] + ' ' +
            self.books['Genres'] + ' ' +
            self.books['Author']
        )


        self.vector = TfidfVectorizer()
        feature_vector = self.vector.fit_transform(books_data_features)


        self.similarity = cosine_similarity(feature_vector)

    def get_recommendations(self, book_title, num_recommendations=10):
        titles_list = self.books['Book'].tolist()


        book_match = difflib.get_close_matches(book_title, titles_list)[0]
        book_index = self.books[self.books.Book == book_match].index[0]


        similarity_score = list(enumerate(self.similarity[book_index]))
        similar_books = sorted(similarity_score, key=lambda x: x[1], reverse=True)[1:num_recommendations+1]

        recommended_books = []
        for book in similar_books:
            index = book[0]
            recommended_books.append({
                'title': self.books.loc[index, 'Book'],
                'author': self.books.loc[index, 'Author'],
                'genres': self.books.loc[index, 'Genres'],
                'rating': self.books.loc[index, 'Avg_Rating'],
                'similarity_score': book[1]
            })

        return recommended_books

def create_recommendation_interface(book_recommendation_system):
    def recommend_books(book_title):
        try:
            recommendations = book_recommendation_system.get_recommendations(book_title)
            recommendation_text = "\n".join([
                f"Title: {rec['title']}\n"
                f"Author: {rec['author']}\n"
                f"Genres: {rec['genres']}\n"
                f"Rating: {rec['rating']:.2f}\n"
                f"Similarity Score: {rec['similarity_score']:.2f}\n"
                "---" for rec in recommendations
            ])
            return recommendation_text
        except Exception as e:
            return f"Error: {str(e)}"

    iface = gr.Interface(
        fn=recommend_books,
        inputs=gr.Textbox(label="Enter a Book Title"),
        outputs=gr.Textbox(label="Recommended Books"),
        title="📚 Book Recommendation System",
        description="Find similar books based on content similarity!"
    )

    return iface

path = kagglehub.dataset_download("ishikajohari/best-books-10k-multi-genre-data")



books = pd.read_csv(path + "/goodreads_data.csv")
recommendation_system = BookRecommendationSystem(books)
interface = create_recommendation_interface(recommendation_system)
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8022129c454dcd376d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
